In [1]:
from pathlib import Path

data_path = Path.cwd() / "lionagi_data"  # Path to the data directory

In [2]:
import lionagi as li

### Prepare QA Tool

In [3]:
docs = li.load(input_dir=data_path, recursive=True, required_exts=[".py"], )
docs = [doc for doc in docs if len(doc.content) > 50]

pile = li.chunk(docs=docs, chunk_size=2000, overlap=0.1,)

In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingModelType
from llama_index.core.postprocessor import LLMRerank

embed_model = OpenAIEmbedding(model=OpenAIEmbeddingModelType.TEXT_EMBED_3_SMALL)

pile.setup_query_engine(
    index_type="llama_index",
    embed_model=embed_model, 
    postprocessor=LLMRerank()
)

In [5]:
await pile.query_pile("what is lionagi")

'lionagi is a collection of libraries and abstract base classes used for various functionalities within a system or application. It includes modules for system utilities, asynchronous operations, data manipulation, API handling, validation functions, and abstract base classes for defining components and concepts within the system.'

In [6]:
async def query_codebase(query: str):
    """
    Perform a query to a QA bot with access to a vector 
    index built with package lionagi codebase

    Args:
        query: The query string to search for in the codebase.

    Returns:
        str: response content from the codebase query.
    """
    return await pile.query_pile(query)

### Construct Workflow

In [7]:
instruction = """
write a good API documentation for this code, must use 
query engine to check meanings of related code concepts 
to accurately describe, for example if a name is used 
but not present in context, you must use the query engine.
make sure to cross reference the code with the query 
engine to ensure the documentation is accurate
"""

edit = """
provide documentation only: final documentation in md 
format of the module of interest, do not include other 
fields do not present in JSON format, only markdown format 
you asked a lot of good questions and got plenty answers, 
please integrate your conversation, be a lot more technical, you will 
be rewarded with 500 dollars for great work, and 
punished for subpar work, take a deep breath, you can do it
"""

In [8]:
from PROMPTS import sys_prompt  # put your system prompt here

tools = li.func_to_tool(query_codebase)

model = li.iModel(
    model="gpt-4o",
    provider="openai",
    interval_tokens=1_000_000,
    interval_requests=1_000,
    interval=60,
)

async def write_doc(context):
    branch = li.Branch(system=sys_prompt, tools=query_codebase, imodel=model)

    form = await branch.direct(
        instruction=instruction,
        context=context,
        reason=True,
        score=True,
        allow_action=True,
        allow_extension=True,
        max_extension=2,
    )
    
    if form is None:
        print("No form returned")
        return None, None

    form.answer = await branch.chat(
        instruction=edit,
        temperature=0.5,
    )
    
    # save all messages into a unique file
    df = branch.to_df()
    df.to_csv(f"lion_doc_{branch.ln_id[:8]}.csv", index=False)

    return form, branch


contexts = [i.content for i in docs]

### Run workflow

In [9]:
form, branch = await write_doc(contexts[80])

In [10]:
form.display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: 
write a good API documentation for this code, must use 
query engine to check meanings of related code concepts 
to accurately describe, for example if a name is used 
but not present in context, you must use the query engine.
make sure to cross reference the code with the query 
engine to ensure the documentation is accurate

- Additional context: """ Copyright 2024 HaiyangLi Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at   http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License. """ from typing import TypeVar, Generic from .abc import Element, Field, Sendable from .pile import Pile, pile from .progression import Progression, progression T = TypeVar("T")  class Exchange(Element, Generic[T]):   """   Item exchange system designed to handle incoming and outgoing flows of items   """   pile: Pile[T] = Field(     default_factory=lambda: pile(),     description="The pile of items in the exchange.",     title="pending items",   )   pending_ins: dict[str, Progression] = Field(     default_factory=dict,     description="The pending incoming items to the exchange.",     title="pending incoming items",     examples=["{ sender_id : Progression}"],   )   pending_outs: Progression = progression()   def __contains__(self, item):     return item in self.pile   def exclude(self, item):     return (       self.pile.exclude(item)       and all([v.exclude(item) for v in self.pending_ins.values()])       and self.pending_outs.exclude(item)     )   def include(self, item, direction):     if self.pile.i...

**reason**: 
 Let's think step by step. To create accurate API documentation, we need to understand the code fully. Some concepts and classes like `Element`, `Field`, `Sendable`, `Pile`, `pile`, `Progression`, and `progression` are not defined in the provided code snippet. We need to query the codebase to get their definitions and understand their roles in the `Exchange` class.

**actions**: 
 
 
1. **query_codebase**(query: Element), 
 
2. **query_codebase**(query: Field), 
 
3. **query_codebase**(query: Sendable), 
 
4. **query_codebase**(query: Pile), 
 
5. **query_codebase**(query: pile), 
 
6. **query_codebase**(query: Progression), 
 
7. **query_codebase**(query: progression)

**action_required**: True

**extension_required**: True

**score**: 8

**action_response**: 
 
 
1. **query_codebase**(query: Element) 
 
 The Element class serves as a base class for elements within the LionAGI system. It includes attributes such as ln_id, a 32-char unique hash identifier, and timestamp, which represents the UTC timestamp of creation., 
 
2. **query_codebase**(query: Field)) = Field, 
 
3. **query_codebase**(query: Sendable)) = Sendable, 
 
4. **query_codebase**(query: Pile) 
 
 Pile is a versatile container class defined in the provided context. It is used for managing collections of Element objects, supporting structured access and manipulation such as retrieval, addition, and deletion of elements. The Pile class facilitates ordered and type-validated storage and access, allowing both index-based and key-based retrieval of items., 
 
5. **query_codebase**(query: pile) 
 
 Pile is a class defined in the provided context information that represents a collection of items. It has methods for validating the items in the pile, converting the pile to a DataFrame, and returning the pile as a list. The pile function defined in the context is a factory function that creates an instance of the Pile class with specified parameters., 
 
6. **query_codebase**(query: Progression) 
 
 Progression is a class that represents a sequence of items in a specific order. It provides methods to add, remove, iterate over, clear, and manipulate items within the progression. The class also includes functionalities to convert the progression to a dictionary or list, extend the progression, include or exclude items, and retrieve items based on index., 
 
7. **query_codebase**(query: progression) 
 
 Progression is a class that represents a sequence of items in a specific order. It provides methods to add, remove, iterate over, clear, and manipulate items within the progression. The class also allows for the creation of progressions with specified orders and names.

**action_performed**: True

**available_tools**: ['query_codebase']

**answer**: 
 ## Class: `Exchange`

**Description**:  
`Exchange` is an item exchange system designed to handle incoming and outgoing flows of items. It manages a collection of items (`pile`), pending incoming items (`pending_ins`), and pending outgoing items (`pending_outs`).

### Attributes:

- **`pile` (Pile[T])**:  
  The pile of items in the exchange. This is a collection managed by the `Pile` class which supports structured access and manipulation of elements.

- **`pending_ins` (dict[str, Progression])**:  
  The pending incoming items to the exchange. This is a dictionary where each key is a sender ID and each value is a `Progression` instance representing the sequence of incoming items from that sender.

- **`pending_outs` (Progression)**:  
  The pending outgoing items from the exchange. This is managed by the `Progression` class which represents a sequence of items in a specific order.

### Methods:

#### `__contains__(self, item)`

**Signature**:
```python
def __contains__(self, item) -> bool:
```

**Parameters**:
- `item`: The item to check for existence in the pile.

**Return Values**:
- `bool`: Returns `True` if the item is in the pile, otherwise `False`.

**Usage Examples**:
```python
exchange = Exchange()
item = ...  # some item
if item in exchange:
    print("Item is in the exchange.")
else:
    print("Item is not in the exchange.")
```

#### `exclude(self, item)`

**Signature**:
```python
def exclude(self, item) -> bool:
```

**Parameters**:
- `item`: The item to exclude from the exchange.

**Return Values**:
- `bool`: Returns `True` if the item was successfully excluded from the pile and all pending lists, otherwise `False`.

**Usage Examples**:
```python
exchange = Exchange()
item = ...  # some item
success = exchange.exclude(item)
if success:
    print("Item excluded successfully.")
else:
    print("Failed to exclude item.")
```

#### `include(self, item, direction)`

**Signature**:
```python
def include(self, item, direction) -> bool:
```

**Parameters**:
- `item`: The item to include in the exchange.
- `direction` (str): The direction of the item flow, either "in" or "out".

**Return Values**:
- `bool`: Returns `True` if the item was successfully included in the pile and the specified direction, otherwise `False`.

**Usage Examples**:
```python
exchange = Exchange()
item = ...  # some item
direction = "in"
success = exchange.include(item, direction)
if success:
    print("Item included successfully.")
else:
    print("Failed to include item.")
```

#### `to_dict(self)`

**Signature**:
```python
def to_dict(self) -> dict:
```

**Return Values**:
- `dict`: Returns a dictionary representation of the exchange.

**Usage Examples**:
```python
exchange = Exchange()
exchange_dict = exchange.to_dict()
print(exchange_dict)
```

### Additional Classes and Concepts:

- **`Element`**: The base class for elements within the LionAGI system, including attributes such as `ln_id` (a unique hash identifier) and `timestamp` (creation timestamp).

- **`Field`**: A descriptor for defining attributes in data models, including default values and descriptions.

- **`Sendable`**: Represents items that can be sent within the exchange system.

- **`Pile`**: A versatile container class for managing collections of `Element` objects, supporting structured access and manipulation.

- **`pile`**: A factory function that creates an instance of the `Pile` class.

- **`Progression`**: Represents a sequence of items in a specific order, providing methods to add, remove, and manipulate items.

- **`progression`**: A factory function that creates an instance of the `Progression` class.

In [11]:
branch.to_df()

,ln_id,message_type,timestamp,role,content,metadata,sender,recipient
0,b3f9d9e433c9b8f06264dca5da0fa7fc,System,2024-05-23T02:29:50.717726,system,"{'system_info': ' you are a helpful assistant,...",{'last_updated': {'recipient': '2024-05-23T02:...,system,bd8c19f854ae8aa001056635221ed06f
1,ace1db4e8d4f3299c4808010e3863f30,Instruction,2024-05-23T02:29:50.779740,user,{'instruction': ' ## Task Instructions...,{'last_updated': {'sender': '2024-05-23T02:29:...,user,bd8c19f854ae8aa001056635221ed06f
2,d0b0f3d8dccc81aeb7b0dd2e14a666da,AssistantResponse,2024-05-23T02:29:56.429577,assistant,"{'assistant_response': '```json { ""answer"": ...",{'last_updated': {'sender': '2024-05-23T02:29:...,bd8c19f854ae8aa001056635221ed06f,user
3,0c966480aaa778ef3f168a3f95af47b7,ActionRequest,2024-05-23T02:29:56.432135,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-23T02:2...,bd8c19f854ae8aa001056635221ed06f,2bc1339f0190af59620163dc7a6cbc0c
4,47eea798102f70e7110d8a3360d16206,ActionRequest,2024-05-23T02:29:56.432519,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-23T02:2...,bd8c19f854ae8aa001056635221ed06f,2bc1339f0190af59620163dc7a6cbc0c
5,61013d9cf3d0569acc3eaea3a3694b00,ActionRequest,2024-05-23T02:29:56.432658,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-23T02:2...,bd8c19f854ae8aa001056635221ed06f,2bc1339f0190af59620163dc7a6cbc0c
6,8c2b2e0061cfe8824aa210ef62338114,ActionRequest,2024-05-23T02:29:56.432780,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-23T02:2...,bd8c19f854ae8aa001056635221ed06f,2bc1339f0190af59620163dc7a6cbc0c
7,dbe6b08a0dfbe93a209c61532af06b8f,ActionRequest,2024-05-23T02:29:56.432920,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-23T02:2...,bd8c19f854ae8aa001056635221ed06f,2bc1339f0190af59620163dc7a6cbc0c
8,7692617ecb59c252b31a1cc81f96353a,ActionRequest,2024-05-23T02:29:56.433037,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-23T02:2...,bd8c19f854ae8aa001056635221ed06f,2bc1339f0190af59620163dc7a6cbc0c
9,6eb18790f3622c7ebb995134b454fe42,ActionRequest,2024-05-23T02:29:56.433161,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-23T02:2...,bd8c19f854ae8aa001056635221ed06f,2bc1339f0190af59620163dc7a6cbc0c


In [12]:
msg = branch.messages[2]

In [13]:
form.tool_schema

{'tools': [{'type': 'function',
   'function': {'name': 'query_codebase',
    'description': 'Perform a query to a QA bot with access to a vector',
    'parameters': {'type': 'object',
     'properties': {'query': {'type': 'string',
       'description': 'The query string to search for in the codebase.'}},
     'required': ['query']}}}]}

In [14]:
form

ln_id                                  4c889e7b99048dbdce4be116fccc85b6
created                                      2024-05-23T02:29:50.779462
metadata              {'last_updated': {'input_fields': '2024-05-23T...
content                                                            None
template_name                                             UnitDirective
assignment                                               task -> answer
input_fields                                        [task, tool_schema]
requested_fields      [answer, reason, actions, action_required, ext...
task                  Follow the prompt and provide the necessary ou...
validation_kwargs     {'reason': {}, 'actions': {}, 'action_required...
confidence_score                                                   None
reason                Let's think step by step. To create accurate A...
actions               {'action_1': {'function': 'query_codebase', 'a...
action_required                                                 

In [15]:
len(form.extension_forms)

1

In [16]:
form.extension_forms[0].display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: N/A
- Additional context: N/A
- Reason and prepare actions with GIVEN TOOLS ONLY.
- Allow auto-extension up to another 1 rounds.
- Give a numeric score in [0, 10] and precision of 0 decimal places.


**reason**: 
 Let's think step by step. We have gathered the necessary information about the classes and methods used in the provided code. This includes understanding the roles of `Element`, `Field`, `Sendable`, `Pile`, `pile`, `Progression`, and `progression`. With this information, we can now create accurate and comprehensive API documentation for the `Exchange` class.

**actions**: 

**action_required**: False

**extension_required**: False

**score**: 10

**available_tools**: ['query_codebase']

**answer**: 
 The API documentation for the provided code is as follows:

## Class: `Exchange`

**Description**:
`Exchange` is an item exchange system designed to handle incoming and outgoing flows of items. It manages a collection of items (`pile`), pending incoming items (`pending_ins`), and pending outgoing items (`pending_outs`).

### Attributes:
- `pile` (Pile[T]): The pile of items in the exchange.
- `pending_ins` (dict[str, Progression]): The pending incoming items to the exchange.
- `pending_outs` (Progression): The pending outgoing items from the exchange.

### Methods:

#### `__contains__(self, item)`

**Signature**:
```python
def __contains__(self, item) -> bool:
```

**Parameters**:
- `item`: The item to check for existence in the pile.

**Return Values**:
- `bool`: Returns `True` if the item is in the pile, otherwise `False`.

**Usage Examples**:
```python
exchange = Exchange()
item = ...  # some item
if item in exchange:
    print("Item is in the exchange.")
else:
    print("Item is not in the exchange.")
```

#### `exclude(self, item)`

**Signature**:
```python
def exclude(self, item) -> bool:
```

**Parameters**:
- `item`: The item to exclude from the exchange.

**Return Values**:
- `bool`: Returns `True` if the item was successfully excluded from the pile and all pending lists, otherwise `False`.

**Usage Examples**:
```python
exchange = Exchange()
item = ...  # some item
success = exchange.exclude(item)
if success:
    print("Item excluded successfully.")
else:
    print("Failed to exclude item.")
```

#### `include(self, item, direction)`

**Signature**:
```python
def include(self, item, direction) -> bool:
```

**Parameters**:
- `item`: The item to include in the exchange.
- `direction` (str): The direction of the item flow, either "in" or "out".

**Return Values**:
- `bool`: Returns `True` if the item was successfully included in the pile and the specified direction, otherwise `False`.

**Usage Examples**:
```python
exchange = Exchange()
item = ...  # some item
direction = "in"
success = exchange.include(item, direction)
if success:
    print("Item included successfully.")
else:
    print("Failed to include item.")
```

#### `to_dict(self)`

**Signature**:
```python
def to_dict(self) -> dict:
```

**Return Values**:
- `dict`: Returns a dictionary representation of the exchange.

**Usage Examples**:
```python
exchange = Exchange()
exchange_dict = exchange.to_dict()
print(exchange_dict)
```
